# 필요한 라이브러리 로드

In [ ]:
import numpy as np
import pandas as pd
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, quote_plus, unquote

# 파일 로드


In [ ]:
# 사용할 Association Rules CSV 로드
asso_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/Result_Asso.csv")
asso_df = asso_df.drop('Unnamed: 0', axis=1)

In [ ]:
# 사용할 Clustering PKL 로드
data_cluster = pd.read_pickle("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/game_feature_matrix.pkl")
data_cluster = data_cluster.reset_index()

# 추천을 받고 싶은 사용자가 보유한 게임 크롤링

In [ ]:
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, quote_plus, unquote

In [ ]:
# 추천을 받고 싶어하는 사용자 Steam64ID 입력
user_to_scan = 76561198143460429

In [ ]:
# API 호출 양식
api_link = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
api_key = '...'
steamid = '76561198003081163' # 스트리머 우왁굳
queryParams = '?' + urlencode({quote_plus('key') : api_key,
                               quote_plus('steamid') : steamid,
                               quote_plus('relationship') : 'friend'
                               })
url = api_link + queryParams

In [ ]:
# 해당 사용자의 게임 플레이 목록 초기화
played_game_list = []

# 쿼리 준비
queryParams = '?' + urlencode({quote_plus('key') : api_key,
                              quote_plus('steamid') : user_to_scan,
                              quote_plus('include_played_free_games') : 1,
                              quote_plus('format') : 'json'
                              })
url = api_link + queryParams

# 데이터를 요청하고 변환 : 스캔한 사용자 프로필이 Public이 아닌 경우 401 에러에 대한 예외 처리
try:
    text = urllib.request.urlopen(url).read().decode('utf-8')
except:
    text = '{}'
json_return = json.loads(text)

# 데이터 해석 작업
if json_return.get('response') != {}:

    # 해당 유저의 게임 목록 가져오기
    owned_game_list = json_return.get('response').get('games')
    # 유효한 게임만 관련 정보 가져오기
    for game in owned_game_list:
        game_info = {}
        if game.get('playtime_forever') < 121:
            continue
        played_game_list.append(game.get('appid'))

In [ ]:
# 사용자가 보유한 게임 중 120분 이상 플레이한 게임 목록 확인
print(played_game_list)

[4000, 550, 105600, 49520, 251570, 108600, 271590, 285900, 286160, 304930, 356670, 361420, 322330, 381210, 386940, 413150, 239140, 431960, 433340, 438100, 477160, 527230, 578080, 648800, 648120, 218620, 658850, 665360, 466240, 739630, 774861, 880940, 897450, 962130, 1049590, 1172470, 1217060, 359550, 1377380, 1404210, 1426210, 1469280, 1478220, 843380, 1531430, 1568590, 1621690, 924970, 1726190, 1794680]


# 클러스터를 통한 동일 클러스터 내 매칭 추천

In [ ]:
# 추천을 받고 싶어하는 사용자의 게임
# ...... 단, 게임 목록은 사용자 주관에 따르나, 가능한 플레이 타임 2시간 이상으로 추천
# ...... 목록이 오류 등으로 인해 비어있다면 아래의 초기값으로 설정됨

want_recommend_games = [281990, 1049590, 1158310, 294100, 1172470]
if played_game_list is not []:
    want_recommend_games = played_game_list

In [ ]:
# 사용자 플레이 게임 목록 기반 클러스터 분석 실시
from collections import Counter

# 클러스터 변환 예시
recommend_cluster = []
for game in want_recommend_games:
    if not data_cluster[data_cluster['GameID'] == str(game)]['cluster'].empty:
        recommend_cluster.append(data_cluster[data_cluster['GameID'] == str(game)]['cluster'].values[0])

def find_mode(lst):
    # 빈 리스트일 경우 예외 처리
    if not lst:
        return []

    # 각 원소의 빈도를 계산
    counts = Counter(lst)
    # 빈도수가 가장 큰 값 찾기
    max_count = max(counts.values())
    # 최빈값(들) 찾기
    modes = [key for key, value in counts.items() if value == max_count]

    return modes

# 사용자 클러스터링 결과 표시
print(find_mode(recommend_cluster))

[4]


In [ ]:
# 추천 알고리즘
#  1. 먼저 사용자가 제출한 게임 목록의 클러스터 확인
#  2. 동일 클러스터 내의 게임으로 AR 필터링
#  3. 보유 게임이 포함된 Row만 살려서 상위 Lift 순으로 추천

filtered_df = asso_df
df_con = pd.DataFrame(columns=['antecedent', 'consequent', 'confidence', 'lift', 'support'])
df_con2 = pd.DataFrame(columns=['antecedent', 'consequent', 'confidence', 'lift', 'support'])
df_ant = pd.DataFrame(columns=['antecedent', 'consequent', 'confidence', 'lift', 'support'])

recommend_cluster = []
for game in want_recommend_games:
    if not data_cluster[data_cluster['GameID'] == str(game)]['cluster'].empty:
        recommend_cluster.append(data_cluster[data_cluster['GameID'] == str(game)]['cluster'].values[0])
recommend_cluster = find_mode(recommend_cluster)

for i in range(len(asso_df['consequent'])):
    con_cell = json.loads(asso_df['consequent'][i])
    for j in range(len(con_cell)):
        result_row = data_cluster.loc[data_cluster['GameID'] == str(con_cell[j])]
        if not result_row.empty:
            if result_row['cluster'].values[0] in recommend_cluster:
                drop_row = asso_df.loc[i].to_frame().T
                df_con = pd.concat([df_con, drop_row])

for i in range(len(df_con['consequent'])):
    con_cell = json.loads(df_con['consequent'].values[i])
    for j in range(len(con_cell)):
        result_row = data_cluster.loc[data_cluster['GameID'] == str(con_cell[j])]
        if not result_row.empty:
            if result_row['cluster'].values[0] in played_game_list:
                drop_row = df_con.loc[i].to_frame().T
                df_con2 = pd.concat([df_con, drop_row])

for i in range(len(df_con['antecedent'])):
    match_count = 0
    con_cell = json.loads(df_con['antecedent'].values[i])
    for j in range(len(con_cell)):
        result_row = data_cluster[data_cluster['GameID'] == con_cell[j]]
        if not result_row.empty:
            if result_row['cluster'].values[0] in played_game_list:
                match_count = match_count + 1
                continue
    if match_count != 0:
        drop_row = df_con.loc[i].to_frame().T
        df_ant = pd.concat([df_con, drop_row])

df_output = pd.merge(df_ant, df_con2)

# AppID - 게임이름 매칭

In [ ]:
# 매칭에 사용할 DB CSV 로드
games_db_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DM2302/Project/SteamGamesDB.csv")

In [ ]:
# FI와 AR 내 게임ID를 게임이름으로 변경

asso_01_pd = df_con.reset_index()

for i in range(len(asso_01_pd['antecedent'])):
    con_cell = json.loads(asso_01_pd['antecedent'][i])
    result_names = []
    for j in range(len(con_cell)):
        result_row = games_db_df[games_db_df['AppID'] == con_cell[j]]
        if not result_row.empty:
            result_name = result_row['Name'].values[0]
            result_names.append(result_name)
    asso_01_pd.loc[i, 'antecedent'] = str(result_names)

for i in range(len(asso_01_pd['consequent'])):
    con_cell = json.loads(asso_01_pd['consequent'][i])
    for j in range(len(con_cell)):
        result_row = games_db_df[games_db_df['AppID'] == con_cell[j]]
        if not result_row.empty:
            result_name = []
            result_name.append(result_row['Name'].values[0])
            asso_01_pd.loc[i, 'consequent'] = str(result_name)

In [ ]:
# 최종 결과 출력 : 해석은 상위 Lift일수록 유사도 높음
# 추가 개선 방향 : Antecedent 혹은 Consequent에 사용자가 플레이한 게임이 하나라도 포함되어 있어야 목록에 포함되도록 개선 가능
asso_01_pd.sort_values(by='lift' ,ascending=False).head(10)

,index,antecedent,consequent,confidence,lift,support
83,11624,['DARK SOULS™ III'],['ELDEN RING'],0.538961,3.243325,0.094083
30,4379,['Starbound'],['Stardew Valley'],0.59596,2.71568,0.080254
58,8749,"[""Don't Starve Together"", 'Terraria']",['Stardew Valley'],0.553134,2.520529,0.092043
33,4917,"['Stardew Valley', 'Terraria']","[""Don't Starve Together""]",0.622699,2.472302,0.092043
82,11498,['Middle-earth™: Shadow of Mordor™'],['The Witcher® 3: Wild Hunt'],0.596947,2.465479,0.088642
20,3443,"['The Elder Scrolls V: Skyrim', 'Borderlands 2']",['The Witcher® 3: Wild Hunt'],0.590327,2.43814,0.094083
63,9048,"['Stardew Valley', 'PAYDAY 2']","[""Don't Starve Together""]",0.608547,2.416112,0.080707
31,4380,['Starbound'],"[""Don't Starve Together""]",0.607744,2.412925,0.081841
22,3506,"['The Elder Scrolls V: Skyrim', 'Portal 2']",['The Witcher® 3: Wild Hunt'],0.580745,2.398565,0.084788
38,5365,"['The Elder Scrolls V: Skyrim', ""Sid Meier's C...",['The Witcher® 3: Wild Hunt'],0.579288,2.392546,0.081161
